In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SAM 3 によるビデオオブジェクトセグメンテーション (Video object segmentation with SAM 3)

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。

In [ ]:
# Google Colabで実行しているかチェック
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except:
    IN_COLAB = False
    print("ローカル環境で実行中")

In [ ]:
# Google Colabの場合、必要なパッケージをインストール
if IN_COLAB:
    !pip install -q pycocotools
    # SAM3のインストール (GitHubリポジトリから)
    !pip install -q git+https://github.com/facebookresearch/sam3.git
    print("パッケージのインストールが完了しました")

In [ ]:
# Google Driveをマウント（データをドライブに保存している場合）
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Driveがマウントされました")
    print("データのパスは /content/drive/MyDrive/... のように指定してください")

### データのアップロード方法

Google Colabでデータを使用する方法は3つあります:

1. **Google Driveを使用**: 上のセルでマウント後、`/content/drive/MyDrive/` 以下のパスを指定
2. **ファイルを直接アップロード**: 左側のファイルアイコンから手動アップロード
3. **Colabのファイルアップロードウィジェットを使用**: 以下のコードを実行

```python
from google.colab import files
uploaded = files.upload()  # ファイル選択ダイアログが表示されます
```

このノートブックでは、`Sam3TrackerPredictor` クラスを使用して、SAM 3 をビデオオブジェクトセグメンテーションに使用する方法を示します。

このノートブックは、インタラクティブなビデオセグメンテーションのための SAM 2 API に従っています。

<a target="_blank" href="https://colab.research.google.com/github/facebookresearch/sam3/blob/main/notebooks/sam3_for_sam2_video_task_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 環境設定 (Environment Set-up)

まず、リポジトリの [インストール手順](https://github.com/facebookresearch/sam3?tab=readme-ov-file#installation) に従って、環境に `sam3` をインストールしてください。

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

## セットアップ (Set-up)

In [ ]:
import torch

# 計算に使用するデバイスを選択
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # ノートブック全体でbfloat16を使用
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # Ampere GPU向けにtfloat32を有効化 (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 3 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

In [ ]:
import glob
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np

import sam3
import torch
from PIL import Image
from sam3.visualization_utils import show_box, show_mask, show_points

# 軸タイトルのフォントサイズ
plt.rcParams["axes.titlesize"] = 12
plt.rcParams["figure.titlesize"] = 12

sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")

### SAM 3 追跡予測器の読み込み (Loading the SAM 3 tracking predictor)

In [ ]:
from sam3.model_builder import build_sam3_video_model

sam3_model = build_sam3_video_model()
predictor = sam3_model.tracker
predictor.backbone = sam3_model.detector.backbone

#### 推論状態の初期化 (Initialize the inference state)

SAM 2 と同様に、SAM 3 もインタラクティブなビデオセグメンテーションのためにステートフルな推論を必要とするため、このビデオ上で **推論状態 (inference state)** を初期化する必要があります。

初期化中に、`video_path` 内のすべての JPEG フレームを読み込み、それらのピクセルを `inference_state` に保存します (以下のプログレスバーで表示されます)。

In [ ]:
video_path = f"{sam3_root}/assets/videos/bedroom.mp4"
inference_state = predictor.init_state(video_path=video_path)

### 例 1: 1つのオブジェクトのセグメンテーションと追跡 (Example 1: Segment & track one object)

注: この `inference_state` を使用して以前に追跡を実行したことがある場合は、まず `clear_all_points_in_video` を介してリセットしてください。

(以下のセルは説明用であり、この `inference_state` は上で新しく初期化されたばかりなので、ここで `clear_all_points_in_video` を呼び出す必要はありません。)

In [ ]:
predictor.clear_all_points_in_video(inference_state)

#### ステップ 1: フレームに最初のクリックを追加する (Step 1: Add a first click on a frame)

手始めに、左側の子供をセグメント化してみましょう。

ここでは、(x, y) = (210, 350) にラベル `1` で **ポジティブクリック** を行います。座標とラベルを `add_new_points` API に送信します。

注: ラベル `1` は *ポジティブクリック (領域を追加)* を示し、ラベル `0` は *ネガティブクリック (領域を削除)* を示します。

In [ ]:
# 視覚化のためにフレームを読み込む
cap = cv2.VideoCapture(video_path)
video_frames_for_vis = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    video_frames_for_vis.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()
frame0 = video_frames_for_vis[0]

width, height = frame0.shape[1], frame0.shape[0]

In [ ]:
ann_frame_idx = 0  # インタラクトするフレームインデックス
ann_obj_id = 1  # インタラクトする各オブジェクトに一意のIDを与える (任意の整数で可)

# 手始めに (x, y) = (210, 350) にポジティブクリックを追加
points = np.array([[210, 350]], dtype=np.float32)
# ラベルについて、`1` はポジティブクリック、`0` はネガティブクリックを意味する
labels = np.array([1], np.int32)

rel_points = [[x / width, y / height] for x, y in points]

points_tensor = torch.tensor(rel_points, dtype=torch.float32)
points_labels_tensor = torch.tensor(labels, dtype=torch.int32)

_, out_obj_ids, low_res_masks, video_res_masks = predictor.add_new_points(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points_tensor,
    labels=points_labels_tensor,
    clear_old_points=False,
)

# 現在の (インタラクトした) フレーム上の結果を表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(frame0)
show_points(points, labels, plt.gca())
show_mask((video_res_masks[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])

#### ステップ 2: 2回目のクリックを追加して予測を修正する (Step 2: Add a second click to refine the prediction)

左側の子供をセグメント化したかったのですが、モデルはショートパンツのマスクのみを予測しているようです。これは、ターゲットオブジェクトが何であるかについて、1回のクリックでは曖昧さが生じるために起こり得ます。子供のシャツにもう一度ポジティブクリックを行うことで、このフレーム上のマスクを修正できます。

ここでは、マスクを拡張するために、(x, y) = (250, 220) にラベル `1` で **2回目のポジティブクリック** を行います。

注: `add_new_points` を呼び出すときは、(最後のクリックだけでなく) **すべてのクリックとそのラベル** を送信する必要があります。

In [ ]:
ann_frame_idx = 0  # インタラクトするフレームインデックス
ann_obj_id = 1  # インタラクトする各オブジェクトに一意のIDを与える (任意の整数で可)

# マスクを修正するために (x, y) = (250, 220) に2回目のポジティブクリックを追加
# すべてのクリック (とそのラベル) を `add_new_points_or_box` に送信する
points = np.array([[210, 350], [250, 220]], dtype=np.float32)
# ラベルについて、`1` はポジティブクリック、`0` はネガティブクリックを意味する
labels = np.array([1, 1], np.int32)

rel_points = [[x / width, y / height] for x, y in points]

points_tensor = torch.tensor(rel_points, dtype=torch.float32)
points_labels_tensor = torch.tensor(labels, dtype=torch.int32)

_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points_tensor,
    labels=points_labels_tensor,
    clear_old_points=False,
)

# 現在の (インタラクトした) フレーム上の結果を表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(frame0)
show_points(points, labels, plt.gca())
show_mask((video_res_masks[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])

この2回目の修正クリックにより、フレーム0上の子供全体のセグメンテーションマスクが得られました。

#### ステップ 3: プロンプトを伝播させてビデオ全体でマスクレットを取得する (Step 3: Propagate the prompts to get the masklet across the video)

ビデオ全体を通してマスクレットを取得するために、`propagate_in_video` API を使用してプロンプトを伝播させます。

In [ ]:
# ビデオ全体で伝播を実行し、結果を辞書に収集する
video_segments = {}  # video_segments はフレームごとのセグメンテーション結果を含む
for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(inference_state, start_frame_idx=0, max_frame_num_to_track=240, reverse=False, propagate_preflight=True):
    video_segments[frame_idx] = {
        out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

# 数フレームごとにセグメンテーション結果を描画
vis_frame_stride = 30
plt.close("all")
for out_frame_idx in range(0, len(video_frames_for_vis), vis_frame_stride):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(video_frames_for_vis[out_frame_idx])
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

#### ステップ 4: 新しいプロンプトを追加してマスクレットをさらに修正する (Step 4: Add new prompts to further refine the masklet)

上記の出力マスクレットを見ると、フレーム150の境界の詳細にいくつかの小さな欠陥があるようです。

SAM 3 を使用すると、モデルの予測をインタラクティブに修正できます。このフレームの (x, y) = (82, 415) にラベル `0` で **ネガティブクリック** を追加して、マスクレットを修正できます。ここでは、修正したいフレームインデックスを示すために、異なる `frame_idx` 引数を使用して `add_new_points_or_box` API を呼び出します。

In [ ]:
ann_frame_idx = 150  # このフレームの詳細をさらに修正する
ann_obj_id = 1  # インタラクトするオブジェクトに一意のIDを与える (任意の整数で可)

# さらなる修正前のセグメントを表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx} -- before refinement")
plt.imshow(video_frames_for_vis[ann_frame_idx])
show_mask(video_segments[ann_frame_idx][ann_obj_id], plt.gca(), obj_id=ann_obj_id)

# セグメントを修正するために、このフレームの (x, y) = (82, 415) にネガティブクリックを追加
points = np.array([[82, 410]], dtype=np.float32)
# ラベルについて、`1` はポジティブクリック、`0` はネガティブクリックを意味する
labels = np.array([0], np.int32)

rel_points = [[x / width, y / height] for x, y in points]

points_tensor = torch.tensor(rel_points, dtype=torch.float32)
points_labels_tensor = torch.tensor(labels, dtype=torch.int32)

_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points_tensor,
    labels=points_labels_tensor,
    clear_old_points=False,
)


# さらなる修正後のセグメントを表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx} -- after refinement")
plt.imshow(video_frames_for_vis[ann_frame_idx])
show_points(points, labels, plt.gca())
show_mask((video_res_masks > 0.0).cpu().numpy(), plt.gca(), obj_id=ann_obj_id)

#### ステップ 5: プロンプトを (再度) 伝播させてビデオ全体でマスクレットを取得する (Step 5: Propagate the prompts (again) to get the masklet across the video)

ビデオ全体の更新されたマスクレットを取得しましょう。ここでは、上記の新しい修正クリックを追加した後、すべてのプロンプトを伝播させるために、再度 `propagate_in_video` を呼び出します。

In [ ]:
# ビデオ全体で伝播を実行し、結果を辞書に収集する
video_segments = {}  # video_segments はフレームごとのセグメンテーション結果を含む
for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(inference_state, start_frame_idx=0, max_frame_num_to_track=300, reverse=False, propagate_preflight=True):
    video_segments[frame_idx] = {
        out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

# 数フレームごとにセグメンテーション結果を描画
vis_frame_stride = 30
plt.close("all")
for out_frame_idx in range(0, len(video_frames_for_vis), vis_frame_stride):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(video_frames_for_vis[out_frame_idx])
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

セグメントはすべてのフレームで良好に見えます。

### 例 2: ボックスプロンプトを使用してオブジェクトをセグメント化する (Example 2: Segment an object using box prompt)

注: この `inference_state` を使用して以前に追跡を実行したことがある場合は、まず `clear_all_points_in_video` を介してリセットしてください。

In [ ]:
predictor.clear_all_points_in_video(inference_state)

クリックを入力として使用することに加えて、SAM 3 は **バウンディングボックス** を介したビデオ内のオブジェクトのセグメンテーションと追跡もサポートしています。

以下の例では、`add_new_points_or_box` API への入力として、フレーム0上の (x_min, y_min, x_max, y_max) = (300, 0, 500, 400) の **ボックスプロンプト** を使用して、右側の子供をセグメント化します。

In [ ]:
ann_frame_idx = 0  # インタラクトするフレームインデックス
ann_obj_id = 4  # インタラクトする各オブジェクトに一意のIDを与える (任意の整数で可)

# 手始めに (x_min, y_min, x_max, y_max) = (300, 0, 500, 400) にボックスを追加
box = np.array([[300, 0, 500, 400]], dtype=np.float32)

rel_box = [[xmin / width, ymin / height, xmax / width, ymax / height] for xmin, ymin, xmax, ymax in box]
rel_box = np.array(rel_box, dtype=np.float32)

_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    box=rel_box,
)

# 現在の (インタラクトした) フレーム上の結果を表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(video_frames_for_vis[ann_frame_idx])
show_box(box[0], plt.gca())
show_mask((video_res_masks[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=ann_obj_id)

ここでは、入力バウンディングボックスがオブジェクトの周りに完全に密着していなくても、SAM 3 は子供全体のかなり良好なセグメンテーションマスクを取得しています。

前の例と同様に、ボックスプロンプトを使用したときに返されたマスクが完璧でない場合は、ポジティブクリックまたはネガティブクリックを使用して出力をさらに **修正** することもできます。これを説明するために、ここでは (x, y) = (460, 60) にラベル `1` で **ポジティブクリック** を行い、子供の髪の周りのセグメントを拡張します。

注: ボックスプロンプトからのセグメンテーションマスクを修正するには、`add_new_points_or_box` を呼び出すときに、**元のボックス入力とその後のすべての修正クリックとそのラベルの両方** を送信する必要があります。

In [ ]:
ann_frame_idx = 0  # インタラクトするフレームインデックス
ann_obj_id = 4  # インタラクトする各オブジェクトに一意のIDを与える (任意の整数で可)

# マスクを修正するために (x, y) = (460, 60) にポジティブクリックを追加
points = np.array([[460, 60]], dtype=np.float32)
# ラベルについて、`1` はポジティブクリック、`0` はネガティブクリックを意味する
labels = np.array([1], np.int32)
# また、新しい修正クリックと一緒に元のボックス入力も
# `add_new_points_or_box` に送信する必要があることに注意してください
box = np.array([[300, 0, 500, 400]], dtype=np.float32)

rel_box = [[xmin / width, ymin / height, xmax / width, ymax / height] for xmin, ymin, xmax, ymax in box]
rel_box = np.array(rel_box, dtype=np.float32)

rel_points = [[x / width, y / height] for x, y in points]

points_tensor = torch.tensor(rel_points, dtype=torch.float32)
points_labels_tensor = torch.tensor(labels, dtype=torch.int32)

_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points_tensor,
    labels=points_labels_tensor,
    box=rel_box,
)

# 現在の (インタラクトした) フレーム上の結果を表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(video_frames_for_vis[ann_frame_idx])
show_box(box[0], plt.gca())
show_points(points, labels, plt.gca())
show_mask((video_res_masks[0][0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])

その後、ビデオ全体を通してマスクレットを取得するために、`propagate_in_video` API を使用してプロンプトを伝播させます。

In [ ]:
# ビデオ全体で伝播を実行し、結果を辞書に収集する
video_segments = {}  # video_segments はフレームごとのセグメンテーション結果を含む
for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(inference_state, start_frame_idx=0, max_frame_num_to_track=300, reverse=False, propagate_preflight=True):
    video_segments[frame_idx] = {
        out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

# 数フレームごとにセグメンテーション結果を描画
vis_frame_stride = 30
plt.close("all")
for out_frame_idx in range(0, len(video_frames_for_vis), vis_frame_stride):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(video_frames_for_vis[out_frame_idx])
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

クリックやボックスに加えて、SAM 3 は `Sam3TrackerPredictor` クラスの `add_new_mask` メソッドを介して、**マスクプロンプト** を入力として直接使用することもサポートしていることに注意してください。これは、例えば半教師ありVOS評価などで役立ちます (例として [tools/vos_inference.py](https://github.com/facebookresearch/sam2/blob/main/tools/vos_inference.py) を参照してください)。

### 例 3: 複数のオブジェクトを同時にセグメント化する (Example 3: Segment multiple objects simultaneously)

注: この `inference_state` を使用して以前に追跡を実行したことがある場合は、まず `clear_all_points_in_video` を介してリセットしてください。

In [ ]:
predictor.clear_all_points_in_video(inference_state)

#### ステップ 1: フレームに2つのオブジェクトを追加する (Step 1: Add two objects on a frame)

SAM 3 は、2つ以上のオブジェクトを同時にセグメント化して追跡することもできます。もちろん、1つずつ行うこともできますが、バッチ処理する方が効率的です (例えば、計算コストを削減するためにオブジェクト間で画像特徴を共有できます)。

今回は、オブジェクトの部分に焦点を当て、このビデオ内の **両方の子供のシャツ** をセグメント化してみましょう。ここでは、これら2つのオブジェクトのプロンプトを追加し、それぞれに一意のオブジェクトIDを割り当てます。

In [ ]:
prompts = {}  # 視覚化のために追加したすべてのクリックを保持する

フレーム0の (x, y) = (200, 300) に **ポジティブクリック** で最初のオブジェクト (左の子供のシャツ) を追加します。

オブジェクトID `2` を割り当てます (任意の整数で構いませんが、追跡する各オブジェクトに対して一意である必要があります)。これは、クリックしているオブジェクトを区別するために `add_new_points_or_box` API に渡されます。

In [ ]:
ann_frame_idx = 0  # インタラクトするフレームインデックス
ann_obj_id = 2  # インタラクトする各オブジェクトに一意のIDを与える (任意の整数で可)

# 最初のオブジェクトを開始するために (x, y) = (200, 300) にポジティブクリックを追加
points = np.array([[200, 300]], dtype=np.float32)
# ラベルについて、`1` はポジティブクリック、`0` はネガティブクリックを意味する
labels = np.array([1], np.int32)
prompts[ann_obj_id] = points, labels

rel_points = [[x / width, y / height] for x, y in points]
points_tensor = torch.tensor(rel_points, dtype=torch.float32)
points_labels_tensor = torch.tensor(labels, dtype=torch.int32)

_, out_obj_ids, low_res_masks, video_res_masks = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points_tensor,
    labels=points_labels_tensor,
)


# 現在の (インタラクトした) フレーム上の結果を表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(video_frames_for_vis[ann_frame_idx])
for i, out_obj_id in enumerate(out_obj_ids):
    show_points(points, labels, plt.gca())
    show_points(*prompts[out_obj_id], plt.gca())
    show_mask((video_res_masks[i][0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_id)

うーん、今回は子供のシャツだけを選択したかったのですが、モデルは子供全体のマスクを予測しています。(x, y) = (275, 175) に **ネガティブクリック** を行って予測を修正しましょう。

In [ ]:
# 最初のオブジェクトを追加
ann_frame_idx = 0  # インタラクトするフレームインデックス
ann_obj_id = 2  # インタラクトする各オブジェクトに一意のIDを与える (任意の整数で可)

# 最初のオブジェクトを修正するために (x, y) = (275, 175) に2回目のネガティブクリックを追加
# すべてのクリック (とそのラベル) を `add_new_points_or_box` に送信する
points = np.array([[200, 300], [275, 175]], dtype=np.float32)
# ラベルについて、`1` はポジティブクリック、`0` はネガティブクリックを意味する
labels = np.array([1, 0], np.int32)
prompts[ann_obj_id] = points, labels

rel_points = [[x / width, y / height] for x, y in points]
points_tensor = torch.tensor(rel_points, dtype=torch.float32)
points_labels_tensor = torch.tensor(labels, dtype=torch.int32)


_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=rel_points,
    labels=points_labels_tensor,
)

# 現在の (インタラクトした) フレーム上の結果を表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(video_frames_for_vis[ann_frame_idx])
for i, out_obj_id in enumerate(out_obj_ids):
    show_points(points, labels, plt.gca())
    show_points(*prompts[out_obj_id], plt.gca())
    show_mask((video_res_masks[i][0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_id)

2回目のネガティブクリックの後、最初のオブジェクトとして左側の子供のシャツを取得できました。

フレーム0の (x, y) = (400, 150) にポジティブクリックを行い、2番目のオブジェクト (右側の子供のシャツ) に進みましょう。ここでは、この2番目のオブジェクトにオブジェクトID `3` を割り当てます (任意の整数で構いませんが、追跡する各オブジェクトに対して一意である必要があります)。

注: 複数のオブジェクトがある場合、`add_new_points_or_box` API は各オブジェクトのマスクのリストを返します。

In [ ]:
ann_frame_idx = 0  # インタラクトするフレームインデックス
ann_obj_id = 3  # インタラクトする各オブジェクトに一意のIDを与える (任意の整数で可)

# 追跡したい2番目のオブジェクトに進みます (オブジェクトID `3` を付与)
# (x, y) = (400, 150) にポジティブクリック
points = np.array([[400, 150]], dtype=np.float32)
# ラベルについて、`1` はポジティブクリック、`0` はネガティブクリックを意味する
labels = np.array([1], np.int32)
prompts[ann_obj_id] = points, labels

rel_points = [[x / width, y / height] for x, y in points]
points_tensor = torch.tensor(rel_points, dtype=torch.float32)
points_labels_tensor = torch.tensor(labels, dtype=torch.int32)


# `add_new_points_or_box` は、このインタラクトしたフレーム上でこれまでに追加されたすべてのオブジェクトのマスクを返します
_, out_obj_ids, low_res_masks, video_res_masks = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    points=points_tensor,
    labels=points_labels_tensor,
)

# 現在の (インタラクトした) フレーム上のすべてのオブジェクトの結果を表示
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(video_frames_for_vis[ann_frame_idx])
for i, out_obj_id in enumerate(out_obj_ids):
    show_points(points, labels, plt.gca())
    show_points(*prompts[out_obj_id], plt.gca())
    show_mask((video_res_masks[i][0] > 0.0).cpu().numpy(), plt.gca(), obj_id=out_obj_id)

今回は、モデルはたった1回のクリックで追跡したいシャツのマスクを予測しました。素晴らしい！

#### ステップ 2: プロンプトを伝播させてビデオ全体でマスクレットを取得する (Step 2: Propagate the prompts to get masklets across the video)

これで、両方のオブジェクトのプロンプトを伝播させて、ビデオ全体を通してそれらのマスクレットを取得します。

注: 複数のオブジェクトがある場合、`propagate_in_video` API は各オブジェクトのマスクのリストを返します。

In [ ]:
# ビデオ全体で伝播を実行し、結果を辞書に収集する
video_segments = {}  # video_segments はフレームごとのセグメンテーション結果を含む
for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(inference_state, start_frame_idx=0, max_frame_num_to_track=300, reverse=False, propagate_preflight=True):
    video_segments[frame_idx] = {
        out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

# 数フレームごとにセグメンテーション結果を描画
vis_frame_stride = 30
plt.close("all")
for out_frame_idx in range(0, len(video_frames_for_vis), vis_frame_stride):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(video_frames_for_vis[out_frame_idx])
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

このビデオでは、両方の子供のシャツがうまくセグメント化されているようです。

さあ、あなた自身のビデオやユースケースで SAM 3 を試してみてください！